<a href="https://colab.research.google.com/github/jugernaut/ProgramacionEnParalelo/blob/desarrollo/Envoltorios/04_TensorFlow2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>TensorFlow</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Profesor: M. en C. Miguel Angel Pérez León</i></h5>
    <h5 align="center"><i>Ayudante: Jesús Iván Coss Calderón</i></h5>
    <h5 align="center"><i>Ayudante: Mario Arturo</i></h5>
  <h5 align="center"><i>Materia: Seminario de programación en paralelo</i></h5>
  </font>

## Introducción

*TensorFlow* es una *API* desarrollado por *Google* y es el conjunto de herramientas libres que se utiliza más ampliamente en el desarrollo de inteligencia artificial.

Existen multiples versiones de *TensorFlow*, sin embargo en escencia vamos a contar con la version 1.x y la versión 2.x. La principal diferencia entre ambas es que la versión 1.x hace uso de **grafos** para representar el flujo de los datos y la versión 2.x se apoya en [Keras](https://enmilocalfunciona.io/deep-learning-basico-con-keras-parte-1/) para generar modelos más intuitivos.

En este documento nos enfocaremos en la versión 2.x de *TensorFlow* con soporte para *GPU's*.

## *Tensor Flow*

Existen varias formas de hacer uso de *TensoFlow*, sin embargo dadas las características del curso, nos vamos a enfocar en la forma declarativa.

Lo primero que necesitamos hacer para acceder a la versión de *TensorFlow* con soporte para GPU's en Google Colab, es desinstalar la versión actual e instalar la versión con soporte para GPU's, además de cambiar el entorno de ejecución del jupyter.

1.   Para cambiar el entorno de ejecución: Primero, ir al menú *Runtime o Entorno de ejecución*, seleccionar *Cambiar tipo de tiempo de ejecución*, y en el cuadro emergente, en *Acelerador de hardware*, seleccione *GPU*, guardamos el cambio y listo.
2.   Posteriormente validamos que se tenga acceso al *GPU*.

In [2]:
import tensorflow as tf

print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.5.0


La celda superior nos indica que tenemos acceso al GPU's y que harémos uso de la versión 2.5.0 de *TensoFlow*.

### *FrameWork*

Inicialmente TensorFlow fue diseñado para hacer uso de grafos para representar los datos y las operaciones que se realizan sobre los mismos. Parte de esa forma de trabajar aun funciona con la versión 2.x de *TensorFlow* y es buena idea comenzar con la misma.

Como en la mayoria de *FrameWorks*, *TensorFlow* cuenta con multiples elementos que ayudan al programador, algunos de estos elementos son:


*   Constantes.
*   Variables.
*   Tensores.
*   Escalares.





#### Operaciones

Pensemos que, como parte de nuestro modelo necesitamos procesar 2 entradas y devolver un resultado. Esta operación es muy sencilla pero muestra como se debe pensar en el flujo de los datos.


In [ ]:
# se realiza la suma de 3 y 5 haciendo uso de tf y del metodo add
a = tf.add(3, 5)
# mostramos el elemento del grafo llamado a
print(a)
# se muestra el resultado de la operación en el nodo a
print(a.numpy())

tf.Tensor(8, shape=(), dtype=int32)
8


Podemos pensar en esta operación de la siguiente forma.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/main/Imagenes/Envoltorios/sumaTF.png?raw=true" width="700">
</center>

#### Ventaja del grafo

El grafo nos da la ventaja de construir de manera organizada y visual la forma en la que se procesan los datos.

Ahora pensemos que deseamos realizar la siguiente operación. 

$$\left(2\times3\right)^{\left(2+5\right)}$$

¿Cómo se vería este grafo y cómo se escribe esta operación con *TensorFlow*?.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/main/Imagenes/Envoltorios/powTF.png?raw=true" width="700">
</center>

In [ ]:
# Variables de Python
x = 2
y = 3

# Operaciones y grafo de TensorFlow
op1 = tf.add(x, y)         
op2 = tf.multiply(x, y)    
op3 = tf.pow(op2, op1)

# Veamos el nodo op3
print(op3)

# El resultado de dicha operación es
print(op3.numpy())

tf.Tensor(7776, shape=(), dtype=int32)
7776


#### Red Neuronal

Conforme vamos agregando más nodos al grafo, este cada vez se parece más a una red, incluso podemos llegar a un punto en el cual el grafo sea similar a una red neuronal.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/main/Imagenes/Envoltorios/som.gif?raw=true" width="700">
</center>

### Acceso a la *GPU*

En la sección anterior vimos que ya se contaba con acceso a la *GPU*, ahora vamos a ver que tan buena idea es hacer uso de la misma.

Vamos a definir 2 métodos que hagan uso de *TensorFlow*, uno de ellos procesando los datos en la *CPU* y el otro en la *CPU*.

In [3]:
# Biblioteca para medir el tiempo
import timeit

# Validamos que se tenga acceso a la GPU de google colab
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print(
        '\n\nNo se tiene habilitado el acceso a la GPU, revisa la configuracion '
        'del notebook.\n\n')
    raise SystemError('No se cuenta con GPU')

# Metodo que realiza el reduce en la CPU de los valores aleatorios de una matriz
def cpu():
    # con esta linea se procesa el bloque en la CPU
    with tf.device('/cpu:0'):
      # generamos una matriz de 100x100x100 con valore aleatorios entre (0,1)
      random_image_cpu = tf.random.normal((100, 100, 100))
      # mediante tensorflow se realiza el reduce y se devuelve un valor
      return tf.math.reduce_sum(random_image_cpu).numpy()

# Metodo que realiza el reduce en la GPU de los valores aleatorios de una matriz
def gpu():
    # con esta linea se procesa el bloque en la GPU
    with tf.device('/device:GPU:0'):
      # generamos una matriz de 100x100x100 con valore aleatorios entre (0,1)
      random_image_gpu = tf.random.normal((100, 100, 100))
      # mediante tensorflow se realiza el reduce y se devuelve un valor
      return tf.math.reduce_sum(random_image_gpu)
  
# Provemos ambos metodos
cpu()
gpu()

# Se ejecutan ambos algoritmos 10 veces y se muestran los respectivos tiempos
print('Se muestra la suma del tiempo de haber ejecutado estos algoritmos '
      '10 veces.')
# Seccion para la CPU
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
# Seccion para la GPU
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
# Mejora en el tiempo de la GPU respecto a la CPU
print('Mejora en el tiempo de ejecucion de GPU '
      'v.s. CPU: {}x'.format(int(cpu_time/gpu_time)))

Se muestra la suma del tiempo de haber ejecutado estos algoritmos  10 veces.
CPU (s):
0.18778029899999638
GPU (s):
0.004059490999992477
Mejora en el tiempo de ejecucion de GPU v.s. CPU: 46x


### Aplicaciones

El ejemplo anterior solo muestra una pequeña parte de un algoritmo en la cual se puede mejorar en gran medida el desempeño de una red neuronal mediate *TensorFlow* en su versión para *GPU's*.

En gran medida las operaciones dentro de una red neuronal (y en general en el aprendizaje de máquina) pueden ser mejoradas mediante el uso de los *GPU's* disponibles.

En la celda anterior se puede ver de manera clara el por qué el uso de TensorFlow se ha vuelto tan popular, sin embargo no olivdemos que muchos de los procesos llevados a cabo quedan ocultos.

### Red Neuronal al instante

Vamos a 'construir' una red en una celda.

## Referencias

*   https://codesachin.wordpress.com/2015/11/28/self-organizing-maps-with-googles-tensorflow/
*   http://www.saedsayad.com/clustering_som.htm
*   https://www.tensorflow.org/install
*   https://relopezbriega.github.io/blog/2016/06/05/tensorflow-y-redes-neuronales/

